In [1]:
import pandas as pd
import re

In [2]:
data = pd.read_excel('french_data.xlsx')                  # reading data (write file_name here)

In [3]:
data['Message'] = data['Message Text']
data['CaseNumber'] = data['Case Number']
data['PostType'] = data['Brand/Fan']

In [4]:
data

,Unnamed: 0,CaseId,Case Number,Message Text,Brand/Fan,Timestamp,Predicted CSAT,Sentiment,Sentiment-Final,Message Type,...,CSAT Score,Is forced CSAT,Match,Fan message order,Previous CSAT,Previous Match,Difference,Message,CaseNumber,PostType
0,0,0000017f036b3089e4b02481e4052ec4,794002,NaN,brand,2022-02-16 16:04:45.740,NaN,0,Neutral,NaN,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,794002,brand
1,1,0000017f036b3089e4b02481e4052ec4,794002,allô,fan,2022-02-16 16:04:30.630,50.0,0,neutral,Others,...,50.0,0.0,1.0,1,NaN,NaN,0.0,allô,794002,fan
2,2,0000017f036b3089e4b02481e4052ec4,794002,allô,fan,2022-02-16 16:04:40.380,50.0,0,neutral,Others,...,50.0,0.0,1.0,2,50.0,1.0,0.0,allô,794002,fan
3,3,0000017f037bbe52e4b02481e40530b9,794355,NaN,brand,2022-02-16 16:34:06.000,NaN,0,Neutral,NaN,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,794355,brand
4,4,0000017f037bbe52e4b02481e40530b9,794355,oui vous,brand,2022-02-16 16:24:16.600,NaN,0,Neutral,NaN,...,NaN,NaN,NaN,0,NaN,NaN,NaN,oui vous,794355,brand
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32023,32023,0000017f03e06b9ee4b02481e405392c,795808,je vous en prie avant ou quitter je vais vous ...,brand,2022-02-16 18:26:06.790,NaN,0,Neutral,NaN,...,NaN,NaN,NaN,97,58.0,1.0,NaN,je vous en prie avant ou quitter je vais vous ...,795808,brand
32024,32024,0000017f03e06b9ee4b02481e405392c,795808,d'accord,fan,2022-02-16 18:26:10.330,50.0,0,neutral,Others,...,50.0,0.0,1.0,98,58.0,1.0,8.0,d'accord,795808,fan
32025,32025,0000017f03e06b9ee4b02481e405392c,795808,merci,fan,2022-02-16 18:26:12.550,60.0,1,positive,Others,...,60.0,0.0,1.0,99,50.0,1.0,-10.0,merci,795808,fan
32026,32026,0000017f03e06b9ee4b02481e405392c,795808,passez 1 bonne soir et surtout prenez soin de ...,brand,2022-02-16 18:26:12.670,NaN,1,Positive,NaN,...,NaN,NaN,NaN,99,60.0,1.0,NaN,passez 1 bonne soir et surtout prenez soin de ...,795808,brand


In [ ]:
data = data[['Message', 'CaseNumber', 'PostType']]    # getting only required columns

In [ ]:
data

In [ ]:
# converting some message from float type to string
def float_to_string(msg):
    msg = str(msg)
    return msg
data['Message'] = data['Message'].apply(float_to_string)

In [ ]:
# cleaning_data
def remove_noise(msg):
    msg = re.sub('(www[^\s]*) | (https?[^\s]*)', '', msg) # removing links
    msg = re.sub('([^\s]*@[^\s]+)', '', msg)              # removing usernames, emails
    msg = re.sub(r'(\S*)[\d]+[^\s]*', '', msg)            # removing numbers
    msg = re.sub(r'\s+', ' ', msg)                        # removing extra spaces
    msg = re.sub('[\n]', ' ', msg)                        # removing new lines
    msg = re.sub(r'[^\w\s]', '', msg)                     # removing punctuations
    msg = msg.encode('ascii', 'ignore').decode('ascii')   # removing emojis
    return msg
data['Message'] = data['Message'].apply(remove_noise)

In [ ]:
# removing extra spaces
def clean_data(msg):
    if msg == None or msg.strip() == '' or msg.strip() == 'nan':
        return None
    msg = msg.replace('nan', '')
    msg = [i.strip().lower() for i in msg.split(' ')]
    msg = list(filter(('').__ne__, msg))                  # removing all occurence of '' in msg
    msg = ' '.join(msg)
    msg = msg.strip()
    if msg == '':
        msg = None
    return msg
data['Message'] = data['Message'].apply(clean_data)

In [ ]:
# removing NA and resetting index
data.dropna(inplace = True)
data.reset_index(inplace = True, drop=True)

In [ ]:
def add_context(row):
    t = row['PostType']
    text = row['Message']
    if t == 'fan':
        return 'le client a demandé ' + text                  # adding customer asked before every fan message
    return "l'agent a répondu " + text                   # adding executive replied before every brand message
data['Message'] = data.apply(add_context, axis=1)

In [ ]:
data[data['CaseNumber'] == 794355]

In [ ]:
data_final = data.groupby('CaseNumber')['Message'].apply(lambda x : '. '.join(x)) # joining all messages acc. to CaseNumber

In [ ]:
data_final = pd.DataFrame({'Message' : data_final.values, 'CaseNumber' : data_final.index})     

In [ ]:
data_final

In [ ]:
data_final.reset_index(inplace = True, drop=True)

In [ ]:
data_final.to_csv('cleaned_data_french.csv', index=None)   # saving cleaned_data for predictions (write file_name here)